In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 199 kB 55.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=07ad34911f43d7481c4dfdf1fba73583cb022394605f1b74a69117e26a142315
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
import time

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('tree').getOrCreate()
df = spark.read.csv('/content/drive/MyDrive/Datasets/Thesis/dr16.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- objid: double (nullable = true)
 |-- modelMag_u: double (nullable = true)
 |-- modelMag_g: double (nullable = true)
 |-- modelMag_r: double (nullable = true)
 |-- modelMag_i: double (nullable = true)
 |-- modelMag_z: double (nullable = true)
 |-- fiberMag_u: double (nullable = true)
 |-- fiberMag_g: double (nullable = true)
 |-- fiberMag_r: double (nullable = true)
 |-- fiberMag_i: double (nullable = true)
 |-- fiberMag_z: double (nullable = true)
 |-- petroR50_r: double (nullable = true)
 |-- petroR90_r: double (nullable = true)
 |-- petroR50_z: double (nullable = true)
 |-- petroR90_z: double (nullable = true)
 |-- r: double (nullable = true)
 |-- i: double (nullable = true)
 |-- z: double (nullable = true)
 |-- redshift: double (nullable = true)
 |-- zerr: double (nullable = true)
 |-- mmug: double (nullable = true)
 |-- mmgr: double (nullable = true)
 |-- mmri: double (nullable = true)
 |-- mmiz: double (nullable = true)
 |-- mfug: doub

In [5]:
print(df.count(), len(df.columns))

3524477 29


In [6]:
df = df.filter((df.modelMag_u != '-9999') & (df.modelMag_g != '-9999') & (df.modelMag_r != '-9999') & (df.modelMag_i != '-9999') & 
               (df.modelMag_z != '-9999') & (df.fiberMag_u != '-9999') & (df.fiberMag_g != '-9999') & (df.fiberMag_r != '-9999') & 
               (df.fiberMag_i != '-9999') & (df.fiberMag_z != '-9999') & (df.petroR50_r != '-9999') & (df.petroR90_r != '-9999') & 
               (df.petroR50_z != '-9999') & (df.petroR90_z != '-9999') & (df.r != '-9999') & (df.i != '-9999') & (df.z != '-9999') & 
               (df.redshift != '-9999') & (df.zerr != '-9999') & (df.mmug != '-9999') & (df.mmgr != '-9999') & (df.mmri != '-9999') & 
               (df.mmiz != '-9999') & (df.mfug != '-9999') & (df.mfgr != '-9999') & (df.mfri != '-9999') & (df.mfiz != '-9999'))
df.show(5)

+---+--------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+---------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+------------------+
|_c0|               objid|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|petroR90_r|        petroR50_z|petroR90_z|       r|       i|       z| redshift|             zerr|     mmug|              mmgr|     mmri|    mmiz|              mfug|     mfgr|    mfri|              mfiz|
+---+--------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+---------+-----------------+---------+------------------+------

In [7]:
print(df.count(), len(df.columns))

3524477 29


In [8]:
import pyspark.sql.functions as func
df = df.withColumn("redshift", func.round(df["redshift"], 2).cast('integer'))

In [9]:
df = df.drop('objid', '_c0')
df.show(5)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+--------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+------------------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|petroR90_r|        petroR50_z|petroR90_z|       r|       i|       z|redshift|             zerr|     mmug|              mmgr|     mmri|    mmiz|              mfug|     mfgr|    mfri|              mfiz|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+--------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+------------------+
|  21.6

In [10]:
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("objid", monotonically_increasing_id())
df.show(5)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+--------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+------------------+-----+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|petroR90_r|        petroR50_z|petroR90_z|       r|       i|       z|redshift|             zerr|     mmug|              mmgr|     mmri|    mmiz|              mfug|     mfgr|    mfri|              mfiz|objid|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+--------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+---------------

In [11]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz'], outputCol = 'features')

In [12]:
output = assembler.transform(df)

In [13]:
final_df = output.select('features', 'redshift')
final_df.show(3)

+--------------------+--------+
|            features|redshift|
+--------------------+--------+
|[21.63269,21.2791...|       1|
|[19.74829,19.4581...|       2|
|[23.74654,23.1965...|       0|
+--------------------+--------+
only showing top 3 rows



In [14]:
train, test = final_df.randomSplit([0.7, 0.3])

In [15]:
from pyspark.ml import Pipeline
nb = NaiveBayes(labelCol="redshift", featuresCol="features")

In [16]:
start_time = time.time()
model = nb.fit(train)
print("%s seconds" % (time.time() - start_time))

77.6002881526947 seconds


In [17]:
nb_train_prediction = model.transform(train)
nb_train_prediction.show(5)

+--------------------+--------+--------------------+--------------------+----------+
|            features|redshift|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|[11.72647,13.1948...|       0|[-1969.2214872968...|[1.0,3.1009395514...|       0.0|
|[12.48301,13.3513...|       0|[-1763.1436731434...|[1.0,6.6236700318...|       0.0|
|[13.42211,12.3572...|       0|[-1840.0827307220...|[1.0,1.0036965831...|       0.0|
|[13.71563,12.6128...|       0|[-1365.2458015591...|[1.0,1.4388816482...|       0.0|
|[13.7939700000000...|       1|[-1762.6953527623...|[1.0,7.6717490384...|       0.0|
+--------------------+--------+--------------------+--------------------+----------+
only showing top 5 rows



In [18]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'redshift', metricName = 'accuracy')
print('Naive Bayes Training Accuracy:', multi_evaluator.evaluate(nb_train_prediction))
print('Training data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(nb_train_prediction))

Naive Bayes Training Accuracy: 0.7992614453493453
Training data (weightedPrecision):  0.8238000970468656


In [19]:
pred_test = model.transform(test)

In [20]:
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'redshift', metricName = 'accuracy')
print('Naive Bayes Accu:', multi_evaluator.evaluate(pred_test))

Naive Bayes Accu: 0.7992732131181716


In [21]:
print('test data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(pred_test))
print('test data (weightedRecall): ', multi_evaluator.setMetricName('weightedRecall').evaluate(pred_test))

test data (weightedPrecision):  0.8236707202596127
test data (weightedRecall):  0.7992732131181716


# Cascading

In [22]:
final_pred = model.transform(final_df)
final_pred.show(5)

+--------------------+--------+--------------------+--------------------+----------+
|            features|redshift|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|[21.63269,21.2791...|       1|[-1387.1382872196...|[0.00773784242082...|       1.0|
|[19.74829,19.4581...|       2|[-1270.6050339425...|[0.09502300235322...|       1.0|
|[23.74654,23.1965...|       0|[-1444.5555210123...|[0.19560087276239...|       1.0|
|[20.63075,20.2542...|       1|[-1325.0420869968...|[0.10493650625114...|       1.0|
|[21.38126,21.1348...|       1|[-1392.8051701350...|[0.16365379827672...|       1.0|
+--------------------+--------+--------------------+--------------------+----------+
only showing top 5 rows



In [23]:
from pyspark.sql.functions import monotonically_increasing_id
final_pred = final_pred.withColumn("objid", monotonically_increasing_id())
final_pred.show(5)

+--------------------+--------+--------------------+--------------------+----------+-----+
|            features|redshift|       rawPrediction|         probability|prediction|objid|
+--------------------+--------+--------------------+--------------------+----------+-----+
|[21.63269,21.2791...|       1|[-1387.1382872196...|[0.00773784242082...|       1.0|    0|
|[19.74829,19.4581...|       2|[-1270.6050339425...|[0.09502300235322...|       1.0|    1|
|[23.74654,23.1965...|       0|[-1444.5555210123...|[0.19560087276239...|       1.0|    2|
|[20.63075,20.2542...|       1|[-1325.0420869968...|[0.10493650625114...|       1.0|    3|
|[21.38126,21.1348...|       1|[-1392.8051701350...|[0.16365379827672...|       1.0|    4|
+--------------------+--------+--------------------+--------------------+----------+-----+
only showing top 5 rows



In [24]:
print(df.count(), len(df.columns))
new_df = df.join(final_pred, df.objid == final_pred.objid).select(df["*"],final_pred["prediction"])
print(df.count(), len(df.columns))
new_df.show(5)

3524477 28
3524477 28
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+------------------+----------+------------------+--------+--------+--------+--------+-----------------+---------+------------------+------------------+---------+---------+---------+---------+---------+-----+----------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|        petroR90_r|petroR50_z|        petroR90_z|       r|       i|       z|redshift|             zerr|     mmug|              mmgr|              mmri|     mmiz|     mfug|     mfgr|     mfri|     mfiz|objid|prediction|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+------------------+----------+------------------+--------+--------+--------+--------+-----------------+---------+------------------+------------------

In [25]:
#from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz', 'prediction'], outputCol = 'features2')

In [26]:
output = assembler.transform(new_df)

In [27]:
final_df = output.select('features2', 'redshift')
final_df.show(3)

+--------------------+--------+
|           features2|redshift|
+--------------------+--------+
|[21.63269,21.2791...|       1|
|[20.90748,21.2373...|       1|
|[23.24831,21.2138...|       3|
+--------------------+--------+
only showing top 3 rows



In [28]:
train, test = final_df.randomSplit([0.7, 0.3])

In [29]:
nb = NaiveBayes(labelCol="redshift", featuresCol="features2")

In [30]:
model2 = nb.fit(train)

In [31]:
dt_train_prediction2 = model2.transform(train)
print('DT Training Accuracy:', multi_evaluator.evaluate(dt_train_prediction2))
print('Training data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(dt_train_prediction2))

DT Training Accuracy: 0.7993872234011412
Training data (weightedPrecision):  0.8239435379051658


In [32]:
pred_test2 = model2.transform(test)

In [33]:
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'redshift', metricName = 'accuracy')
print('Naive Bayes Accu:', multi_evaluator.evaluate(pred_test2))
print('test data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(pred_test2))

Naive Bayes Accu: 0.7989799612286969
test data (weightedPrecision):  0.8233366311051866


In [34]:
#print('test data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(pred_test2))
#print('test data (weightedRecall): ', multi_evaluator.setMetricName('weightedRecall').evaluate(pred_test2))